# Analysis of Similarities in Threat Actors

## Description 

This notebook takes the analysis from the other notebooks (e.g. [aggregation-notebook](./aggregation-notebook.ipynb), [attack-pattern-similarities](./attack-pattern-similarities.ipynb), [malware-similarities](./malware-similarities.ipynb), etc) and combines these into a structured approach to analyzing the similarities among threat groups. See the [README](./README.md) for the methodology behind this analysis. 

**NOTE:** this notebook assumes that the other notebooks are completed and the files for each similarity matrix exist. 

# Similarity Analysis

In [1]:
import numpy as np
import pandas as pd 
import json 
from functions.functions import generate_similarity_table

## Setup 

**Loading the necessary data**

In [2]:
data_dir:str = 'data/analysis-outcomes/'
actor_names:list[str] = [ a['name'] for a in json.load(open('data/jsons/intrusion-sets.json', 'r')) ]

# Actor TTP similarities
actor_ttps_similarity_matrix:np.matrix = pd.read_csv(data_dir + 'actor-attack-pattern-similarities/actor-ttp-similarity-matrix.csv', header=None).values 
with open(data_dir + 'actor-attack-pattern-similarities/matrix-labels.json', 'r') as file:
    actor_ttps_matrix_labels:dict[str, list[str]] =  json.load(file)
    
# Actor malware similarities
actor_malware_similarity_matrix:np.matrix = pd.read_csv(data_dir + 'actor-malware-similarities/actor-malware-similarity-matrix.csv', header=None).values 
with open(data_dir + 'actor-malware-similarities/matrix-labels.json', 'r') as file: 
    actor_malware_matrix_labels:list[str] = json.load(file)
    
# Actor tools similarities
actor_tools_similarities_matrix:np.matrix = pd.read_csv(data_dir + 'actor-tool-similarities/actor-tool-similarity-matrix.csv', header=None).values 
with open(data_dir + 'actor-tool-similarities/matrix-labels.json', 'r') as file: 
    actor_tools_matrix_labels:list[str] = json.load(file)
    

**Defining hyperparameters**

In [3]:
# NOTE: the following can be defined as fractions or probabilities but MUST add to 1 (100%)

# ttp_similarity_weight := weight for the actor_ttps_similarity_matrix 
ttp_similarity_weight:float = 1/6

# malware_similarity_weight := weight for the actor_malware_similarity_matrix
malware_similarity_weight:float = 1/2

# tool_similarity_weight := weight for the actor_tools_similarity_matrix
tool_similarity_weight:float = 1/3

**Checking that hyperparams are valid**

In [4]:
if ttp_similarity_weight + malware_similarity_weight + tool_similarity_weight != 1: 
    raise ValueError('Hyperparameters do not add up to 1.')

## Creating the model

**Weighting the matrices**

In [5]:
# TTPs similarity matrix
actor_ttps_similarity_matrix = ttp_similarity_weight * actor_ttps_similarity_matrix

# Malware similarity matrix
actor_malware_similarity_matrix = malware_similarity_weight * actor_malware_similarity_matrix

# Tools similarity matrix 
actor_tools_similarities_matrix = tool_similarity_weight * actor_tools_similarities_matrix

**Checking that the matrices have the same shape**

In [6]:
if not (actor_ttps_similarity_matrix.shape == actor_malware_similarity_matrix.shape == actor_tools_similarities_matrix.shape): 
    raise ValueError('Similarity matrices do not have the same shape. Check your matrices.')

**Summing the matrices together**

In [7]:
summed_similarities_matrix:np.matrix = actor_ttps_similarity_matrix + actor_malware_similarity_matrix + actor_tools_similarities_matrix

# Save the summed similarities matrix 
np.savetxt('data/analysis-outcomes/similarity-matrix.csv', summed_similarities_matrix.astype(float), delimiter=',', fmt='%f')

In [8]:
sim_table, pretty_table = generate_similarity_table(summed_similarities_matrix, actor_names, check_if_aliases=True)


# Convert the table to a dataframe and save it
sim_table_df:pd.DataFrame = pd.DataFrame(sim_table, columns=['Actor 1', 'Actor 2', 'Similarity Rating', 'Is Alias'])
sim_table_df.to_csv('data/analysis-outcomes/similarities-table.csv', index=False)